In [2]:
import pandas as pd
import requests
from urllib.parse import quote_plus
import os

#### Reading in and Deduplicating the Primary File by Taxa (probably downloaded from Laminar)

In [3]:
# Specify the input and output files (include the entire path name)
original_file = input("Enter the original CSV file name (with extension). The file path is relative from where this program is run: ")

# Save the original file within a variable called df
original_df = pd.read_csv(original_file)

Enter the original CSV file name (with extension). The file path is relative from where this program is run: ../940324_v1_larval_sebastes_diet_and_morphometric_data1.csv


In [4]:
# Run this block if you want to view the read-in original file
original_df

,Cruise,ISO_DateTime_UTC,ISO_DateTime_PDT,Line,St,Lat,Long,Tow_Depth,Mesh_Size,Larva_ID,...,rad40,rad41,rad42,rad43,rad44,rad45,rad46,rad47,rad48,rad49
0,BBB2104,2021-04-07T17:52Z,2021-04-07T10:52,90.0,35.0,33.249950,-118.254150,29.65,505,RF002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BBB2104,2021-04-07T17:52Z,2021-04-07T10:52,90.0,35.0,33.249950,-118.254150,29.65,505,RF004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BBB2104,2021-04-07T16:15Z,2021-04-07T9:15,90.0,30.0,33.418633,-117.906200,28.91,505,RF011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BBB2104,2021-04-07T16:15Z,2021-04-07T9:15,90.0,30.0,33.418633,-117.906200,28.91,505,RF012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BBB2104,2101-04-06T21:20,2101-04-06T14:20,93.3,35.0,32.848000,-117.535500,29.74,505,RF020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,BBB2104,2021-04-07T21:04Z,2021-04-07T14:04,90.0,37.0,33.183783,-118.388867,30.13,505,RF206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,SW2104,2021-04-15T20:46Z,2021-04-15T13:46,85.0,42.9,33.850000,-119.370000,32.42,505,RF209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,SW2104,2021-04-15T20:46Z,2021-04-15T13:46,85.0,42.9,33.850000,-119.370000,32.42,505,RF211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,SW2104,2021-04-15T20:46Z,2021-04-15T13:46,85.0,42.9,33.850000,-119.370000,32.42,505,RF212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Identify the column containing species names in the original file
while True:
        column_name = input("Enter the name of the column that contains the scientific names: ")
        if column_name in original_df.columns:
            break  # Exit the loop if the column name is valid
        else:
            print(f"Column '{column_name}' not found. Please try again. (make sure capitalization pattern is accurate.)")

Enter the name of the column that contains the scientific names: Species


In [10]:
# Read out a list of deduplicated taxa from df

def get_unique_values(original_df, column_name):
    """Get unique values from a specified DataFrame column."""
    return original_df[column_name].unique()

unique_values = get_unique_values(original_df, column_name)
print(f"Unique values in column '{column_name}':\n{unique_values}")

Unique values in column 'Species':
['Sebastes semicinctus' 'Sebastes macdonaldi' 'Sebastes jordani'
 'Sebastes melanostomus' 'Sebastes simulator' 'Sebastes rufinanus'
 'Sebastes hopkinsi' 'Sebastes mystinus/entomelas' 'Sebastes DNW'
 'Sebastes ensifer' 'Sebastes rufus' 'Sebastes aurora' 'Sebastes miniatus'
 'Sebastes wilsoni' 'Sebastes levis' 'Sebastes goodei'
 'Sebastes paucispinis' 'Sebastes saxicola' 'Sebastes moseri'
 'Sebastes diploproa']


#### Fetching data from the WoRMs API

In [11]:
# Step 5: Fetch data from the API

def fetch_api_data(unique_values):
    """Fetch data from the WoRMS API for a list of unique scientific names."""
    base_url = 'https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames%5B%5D='
    api_results = []

    for each in unique_values:
        try:
            url_sp = quote_plus(each)
            url = f'{base_url}{url_sp}&marine_only=true'
            print("Next api query to run:")
            print(url)

            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            data = response.json()

            for i in data:
                for y in i:
                    y['PI_entered_name'] = each  # Add the scientific name to the result
                    api_results.append(y)

        except requests.exceptions.HTTPError as e:
            print(f"HTTP error: {e}")
            api_results.append({'PI_entered_name': each, 'error_message': 'HTTP error'})
        except Exception as e:
            print(f"Error fetching data: {e}. Likely this means no close match was found.")
            api_results.append({'PI_entered_name': each, 'error_message': 'Error fetching data'})
    print("The program finished processing and printing all API results.")

    return api_results

api_results = fetch_api_data(unique_values)

Next api query to run:
https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames%5B%5D=Sebastes+semicinctus&marine_only=true
The program finished processing and printing all API results.
Next api query to run:
https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames%5B%5D=Sebastes+macdonaldi&marine_only=true
The program finished processing and printing all API results.
Next api query to run:
https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames%5B%5D=Sebastes+jordani&marine_only=true
The program finished processing and printing all API results.
Next api query to run:
https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames%5B%5D=Sebastes+melanostomus&marine_only=true
The program finished processing and printing all API results.
Next api query to run:
https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames%5B%5D=Sebastes+simulator&marine_only=true
The program finished processing and printin

In [12]:
# Convert the API results into a dataframe
api_results_df = pd.DataFrame(api_results)

# Display the API results dataframe within the Jupyter Notebook
## This DF represents all of the WoRMS metadata from the deduplicated Taxa in the original file")
api_results_df

,AphiaID,url,scientificname,authority,status,unacceptreason,taxonRankID,rank,valid_AphiaID,valid_name,...,citation,lsid,isMarine,isBrackish,isFreshwater,isTerrestrial,isExtinct,match_type,modified,PI_entered_name
0,274851,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes semicinctus,"(Gilbert, 1897)",accepted,None,220,Species,274851,Sebastes semicinctus,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274851,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes semicinctus
1,274814,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes macdonaldi,"(Eigenmann & Beeson, 1893)",accepted,None,220,Species,274814,Sebastes macdonaldi,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274814,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes macdonaldi
2,274806,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes jordani,"(Gilbert, 1896)",accepted,None,220,Species,274806,Sebastes jordani,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274806,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes jordani
3,274819,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes melanostomus,"(Eigenmann & Eigenmann, 1890)",accepted,None,220,Species,274819,Sebastes melanostomus,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274819,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes melanostomus
4,274854,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes simulator,"Chen, 1971",accepted,None,220,Species,274854,Sebastes simulator,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274854,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes simulator
5,274846,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes rufinanus,"Lea & Fitch, 1972",accepted,None,220,Species,274846,Sebastes rufinanus,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274846,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes rufinanus
6,274800,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes hopkinsi,"(Cramer, 1895)",accepted,None,220,Species,274800,Sebastes hopkinsi,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274800,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes hopkinsi
7,126175,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes,"Cuvier, 1829",accepted,None,180,Genus,126175,Sebastes,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:126175,1,1,0,0,None,exact,2014-11-21T13:09:37.943Z,Sebastes mystinus/entomelas
8,126175,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes,"Cuvier, 1829",accepted,None,180,Genus,126175,Sebastes,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:126175,1,1,0,0,None,exact,2014-11-21T13:09:37.943Z,Sebastes DNW
9,274790,https://www.marinespecies.org/aphia.php?p=taxd...,Sebastes ensifer,"Chen, 1971",accepted,None,220,Species,274790,Sebastes ensifer,...,"Froese, R. and D. Pauly. Editors. (2025). Fish...",urn:lsid:marinespecies.org:taxname:274790,1,0,0,0,None,exact,2008-01-15T17:27:08.177Z,Sebastes ensifer


#### Saving the deduplicated and merged WoRMs results to a CSV

In [14]:
def save_results_to_csv(results, output_file):
    """Save the results to a CSV file."""
    resulting_df = pd.DataFrame(results)
    resulting_df.to_csv(output_file, index=False)  # Set index=False to avoid writing row numbers

# File name handling
deduplicated_worms_output_file = f'{original_file[:-4]}_deduplicated_worms_taxa_results.csv'
deduplicated_worms_output_file_name = str(deduplicated_worms_output_file)
save_results_to_csv(api_results, deduplicated_worms_output_file)

print(deduplicated_worms_output_file_name + " was saved in the same folder as the input file")


../940324_v1_larval_sebastes_diet_and_morphometric_data1_deduplicated_worms_taxa_results.csv was saved in the same folder as the input file


#### Joining the API results to the original file


In [16]:
# Specify columns to keep from the API df after join is completed

merge_columns = [
        'PI_entered_name', 'AphiaID', 'scientificname',
        'status', 'rank', 'valid_name', 'lsid', 'match_type'
    ]

merged_df = original_df.merge(api_results_df[merge_columns],
                                  left_on=column_name,
                                  right_on='PI_entered_name',
                                  how='left')

## This df shows the merge_columns from the API results df joined to the original_df
merged_df

,Cruise,ISO_DateTime_UTC,ISO_DateTime_PDT,Line,St,Lat,Long,Tow_Depth,Mesh_Size,Larva_ID,...,rad48,rad49,PI_entered_name,AphiaID,scientificname,status,rank,valid_name,lsid,match_type
0,BBB2104,2021-04-07T17:52Z,2021-04-07T10:52,90.0,35.0,33.249950,-118.254150,29.65,505,RF002,...,NaN,NaN,Sebastes semicinctus,274851,Sebastes semicinctus,accepted,Species,Sebastes semicinctus,urn:lsid:marinespecies.org:taxname:274851,exact
1,BBB2104,2021-04-07T17:52Z,2021-04-07T10:52,90.0,35.0,33.249950,-118.254150,29.65,505,RF004,...,NaN,NaN,Sebastes macdonaldi,274814,Sebastes macdonaldi,accepted,Species,Sebastes macdonaldi,urn:lsid:marinespecies.org:taxname:274814,exact
2,BBB2104,2021-04-07T16:15Z,2021-04-07T9:15,90.0,30.0,33.418633,-117.906200,28.91,505,RF011,...,NaN,NaN,Sebastes jordani,274806,Sebastes jordani,accepted,Species,Sebastes jordani,urn:lsid:marinespecies.org:taxname:274806,exact
3,BBB2104,2021-04-07T16:15Z,2021-04-07T9:15,90.0,30.0,33.418633,-117.906200,28.91,505,RF012,...,NaN,NaN,Sebastes semicinctus,274851,Sebastes semicinctus,accepted,Species,Sebastes semicinctus,urn:lsid:marinespecies.org:taxname:274851,exact
4,BBB2104,2101-04-06T21:20,2101-04-06T14:20,93.3,35.0,32.848000,-117.535500,29.74,505,RF020,...,NaN,NaN,Sebastes melanostomus,274819,Sebastes melanostomus,accepted,Species,Sebastes melanostomus,urn:lsid:marinespecies.org:taxname:274819,exact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,BBB2104,2021-04-07T21:04Z,2021-04-07T14:04,90.0,37.0,33.183783,-118.388867,30.13,505,RF206,...,NaN,NaN,Sebastes DNW,126175,Sebastes,accepted,Genus,Sebastes,urn:lsid:marinespecies.org:taxname:126175,exact
76,SW2104,2021-04-15T20:46Z,2021-04-15T13:46,85.0,42.9,33.850000,-119.370000,32.42,505,RF209,...,NaN,NaN,Sebastes semicinctus,274851,Sebastes semicinctus,accepted,Species,Sebastes semicinctus,urn:lsid:marinespecies.org:taxname:274851,exact
77,SW2104,2021-04-15T20:46Z,2021-04-15T13:46,85.0,42.9,33.850000,-119.370000,32.42,505,RF211,...,NaN,NaN,Sebastes semicinctus,274851,Sebastes semicinctus,accepted,Species,Sebastes semicinctus,urn:lsid:marinespecies.org:taxname:274851,exact
78,SW2104,2021-04-15T20:46Z,2021-04-15T13:46,85.0,42.9,33.850000,-119.370000,32.42,505,RF212,...,NaN,NaN,Sebastes jordani,274806,Sebastes jordani,accepted,Species,Sebastes jordani,urn:lsid:marinespecies.org:taxname:274806,exact


#### Saving the merged API results with the original data file as a new CSV

In [18]:
# File name handling

def save_results_to_csv(results, output_file):
    """Save the results to a CSV file."""
    resulting_df = pd.DataFrame(results)
    resulting_df.to_csv(output_file, index=False)  # Set index=False to avoid writing row numbers

merged_worms_output_file = f'{original_file[:-4]}_merged_worms_bcodmo.csv'
merged_worms_output_file_name = str(merged_worms_output_file)
save_results_to_csv(merged_df, merged_worms_output_file)

print(merged_worms_output_file_name + " was saved in the same folder as the input file.")    

../940324_v1_larval_sebastes_diet_and_morphometric_data1_merged_worms_bcodmo.csv was saved in the same folder as the input file.
